In [2]:
# Project Path Setup

import sys
import os
from pathlib import Path

PROJECT_ROOT = Path().resolve().parent
os.chdir(PROJECT_ROOT)
sys.path.insert(0, str(PROJECT_ROOT))

print("✅ PROJECT_ROOT:", PROJECT_ROOT)
print("📂 Working directory:", Path.cwd())
# This script demonstrates how to display the current working directory and its parent directories.

✅ PROJECT_ROOT: C:\Users\rajku\Desktop\enterprise_knowledge_assistant
📂 Working directory: C:\Users\rajku\Desktop\enterprise_knowledge_assistant


In [3]:
# import 

import faiss
import pickle
import subprocess
from typing import List

from sentence_transformers import SentenceTransformer
from langchain_core.documents import Document

from src.vector_store.faiss_store import load_vector_store


In [4]:
# Load Embedding Model

def load_embedding_model(
    model_name: str = "all-MiniLM-L6-v2"
):
    print(f"🔹 Loading embedding model: {model_name}")
    return SentenceTransformer(model_name)


In [5]:
# Load Vector Store (from Step-4)

VECTOR_STORE_PATH = PROJECT_ROOT / "data" / "processed" / "vector_store.pkl"

faiss_index, documents = load_vector_store(VECTOR_STORE_PATH)

print(f"✅ Loaded FAISS index with {faiss_index.ntotal} vectors")


✅ Vector store loaded
✅ Loaded FAISS index with 98 vectors


In [6]:
# Retrieval Function (FAISS → Context)

def retrieve_context(
    query: str,
    model: SentenceTransformer,
    index: faiss.IndexFlatL2,
    documents: List[Document],
    top_k: int = 4
) -> List[str]:

    query_embedding = model.encode(
        [query],
        convert_to_numpy=True
    )

    distances, indices = index.search(query_embedding, top_k)

    contexts = []
    for idx in indices[0]:
        contexts.append(documents[idx].page_content)

    return contexts


In [7]:
# Prompt Builder (THIS MAKES IT RAG)

def build_rag_prompt(
    query: str,
    contexts: List[str]
) -> str:

    context_block = "\n\n".join(
        [f"- {ctx}" for ctx in contexts]
    )

    prompt = f"""
You are a knowledgeable assistant.
Answer the question ONLY using the context below.
If the answer is not present in the context, say "I don't know".

Context:
{context_block}

Question:
{query}

Answer:
"""
    return prompt.strip()


### 📌 This is where augmentation happens
### 📌 This is what prevents hallucination

In [12]:
# Call Local LLM via Ollama

def call_llm_ollama(
    prompt: str,
    model_name: str = "llama3.2:3b"
) -> str:
    """
    Call Ollama LLM using UTF-8 safe subprocess execution.
    """

    result = subprocess.run(
        ["ollama", "run", model_name],
        input=prompt.encode("utf-8"),   # 🔑 FIX HERE
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )

    return result.stdout.decode("utf-8", errors="ignore").strip()



In [13]:
# Load Model

embedding_model = load_embedding_model()


🔹 Loading embedding model: all-MiniLM-L6-v2


In [14]:
# Ask a Question (End-to-End RAG)

query = "How does surface roughness affect heat transfer?"

contexts = retrieve_context(
    query=query,
    model=embedding_model,
    index=faiss_index,
    documents=documents,
    top_k=4
)

prompt = build_rag_prompt(query, contexts)

print("🔹 RAG PROMPT:\n")
print(prompt)


🔹 RAG PROMPT:

You are a knowledgeable assistant.
Answer the question ONLY using the context below.
If the answer is not present in the context, say "I don't know".

Context:
- Thermal behavior analysis
The melting temperature of a solder interconnect is a critical parameter to ensure compatibility with the operating 
temperature of electronic devices and the thermal limits of integrated IC components. It is also imperative to 
assess the influence of sustainable reinforcements such as Agro-waste (RHA) and (WS), which are increasingly 
considered in green manufacturing. Figure 3 presents the differential scanning calorimetry (DSC) profiles of the 
plain and composite solder samples, while the associated thermal parameters, melting temperature (Tm), solidus 
temperature (Ts), liquidus temperature (TL), pasty range (ΔT), and enthalpy of fusion (ΔH) are summarized in 
Table 2.
Compared to the plain solder alloy (S0), which has a Tm of 226.10 °C, both S1 (224.65 °C) and S2 (225.44 °C) 
sho

In [15]:
# Generate Answer (FINAL OUTPUT)

answer = call_llm_ollama(prompt)

print("🧠 RAG ANSWER:\n")
print(answer)


🧠 RAG ANSWER:

I don't know. The provided context discusses the thermal behavior analysis, including melting temperatures, solidus temperatures, liquidus temperatures, pasty ranges, and enthalpy of fusion of different solder alloys, but it doesn't mention how surface roughness affects heat transfer.
